required modules; note that you need an arcpy environment for python to run

In [14]:
import arcpy, string, os, datetime
from arcpy.sa import *

arcpy settings and file locations, due to file size restrictions you need to create a geodatabase named "EHA.gdb" within your working directory and put your region of interest in it

In [15]:
wd = os.getcwd()

arcpy.env.workspace = os.path.join(wd, "EHA.gdb")
rasterSpace = os.path.join(wd, "Rasters")
ndvi_out = os.path.join(wd, "NDVI")
arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("spatial")

'CheckedOut'

In [16]:
rasterFolders = os.listdir(rasterSpace)

In [23]:
ndviFiles = os.listdir(ndvi_out)
#clears the NDVI folder
for file in ndviFiles:
    if file != ".gitignore":
        os.remove(os.path.join(ndvi_out, file))

the name of the feature class of your region from within the geodatabase you created with the name "EHA.gdb"

In [24]:
roi = "keji"

see comments for descriptions in this cell

In [18]:
#initializes a list of shapefiles that will be merged together at the end of the script
shpList = []

#loops through each file placed in the rasters folder
for i in rasterFolders:
    folder = os.path.join(rasterSpace, i)
    insideFolder = os.listdir(folder)
    
    #pulls the image acquisition date from the first file inside of each image folder
    date = insideFolder[0].split("_")[3]
    isodate = datetime.datetime.strptime(date, "%Y%m%d").date().isoformat()
    print(isodate)
    
    #generates a save location for each shapefile generated from the raster
    saveLocation = os.path.join(ndvi_out, (isodate + "_ndviPoint.shp"))
    print(saveLocation)
    
    #identifies red and NIR bands for Landsat 5
    if i.startswith("LT05"):
        print("Landsat 5")
        for file in insideFolder:
            if file.endswith("band3.tif"):
                redRaster = Raster(os.path.join(folder, file))
            if file.endswith("band4.tif"):
                nirRaster = Raster(os.path.join(folder, file))
    
    #identifies red and NIR bands for landsat 8
    if i.startswith("LC08"):
        print("Landsat 8")
        for file in insideFolder:
            if file.endswith("band4.tif"):
                redRaster = Raster(os.path.join(folder, file))
            if file.endswith("band5.tif"):
                nirRaster = Raster(os.path.join(folder, file))
    
    #generates NDVI for the image
    ndvi = (nirRaster - redRaster) / (nirRaster + redRaster)
    
    #extract by mask to main portion of Kejimkujik national park - switch this if you want a different study area
    ndviExtracted = ExtractByMask(ndvi, roi)
    
    #raster to point
    arcpy.RasterToPoint_conversion(ndviExtracted, saveLocation, "VALUE")
    
    #adds date field
    arcpy.AddField_management(saveLocation, "Date", "DATE")
    
    #calculates date field
    arcpy.CalculateField_management(saveLocation, "Date", "'" + isodate + "'")
    
    #add to list of shapefile locations
    shpList.append(saveLocation)
    
#merge the shapefiles together
arcpy.Merge_management(shpList, "mergedNDVI")
    
print("A merged points shapefile is available in the geodatabase.")

2014-04-07
D:\OneDrive\OneDrive - Dalhousie University\JupyterNotebooks\EHA_Paper\NDVI\2014-04-07_ndviPoint.shp
Landsat 8


ModuleNotFoundError: No module named 'geoanalyticssoap'

ModuleNotFoundError: No module named 'ErrorUtils'

ModuleNotFoundError: No module named 'SSDataObject'

ModuleNotFoundError: No module named 'geoanalyticssoap'

ModuleNotFoundError: No module named 'ErrorUtils'

ModuleNotFoundError: No module named 'SSDataObject'

2019-04-05
D:\OneDrive\OneDrive - Dalhousie University\JupyterNotebooks\EHA_Paper\NDVI\2019-04-05_ndviPoint.shp
Landsat 8
2019-05-07
D:\OneDrive\OneDrive - Dalhousie University\JupyterNotebooks\EHA_Paper\NDVI\2019-05-07_ndviPoint.shp
Landsat 8
2013-04-27
D:\OneDrive\OneDrive - Dalhousie University\JupyterNotebooks\EHA_Paper\NDVI\2013-04-27_ndviPoint.shp
Landsat 8
2015-04-01
D:\OneDrive\OneDrive - Dalhousie University\JupyterNotebooks\EHA_Paper\NDVI\2015-04-01_ndviPoint.shp
Landsat 8
2016-05-21
D:\OneDrive\OneDrive - Dalhousie University\JupyterNotebooks\EHA_Paper\NDVI\2016-05-21_ndviPoint.shp
Landsat 8
2018-05-27
D:\OneDrive\OneDrive - Dalhousie University\JupyterNotebooks\EHA_Paper\NDVI\2018-05-27_ndviPoint.shp
Landsat 8
2019-03-27
D:\OneDrive\OneDrive - Dalhousie University\JupyterNotebooks\EHA_Paper\NDVI\2019-03-27_ndviPoint.shp
Landsat 8


KeyboardInterrupt: 